In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime
import itertools
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_Columns', 100)

In [2]:
train_y = pd.read_csv('../input/train.csv')
print(train_y.shape)
train_X = pd.read_csv('../input/train_X.csv')
print(train_X.shape)
test_X = pd.read_csv('../input/test_X.csv')
print(test_X.shape)

(12026, 19)
(12026, 371)
(12008, 372)


In [3]:
n_fold = 5

In [4]:
target = train_y['likes'].apply(np.log1p)

In [5]:
train_X.head()

,description_w2v_0,description_w2v_1,description_w2v_2,description_w2v_3,description_w2v_4,description_w2v_5,description_w2v_6,description_w2v_7,description_w2v_8,description_w2v_9,description_w2v_10,description_w2v_11,description_w2v_12,description_w2v_13,description_w2v_14,description_w2v_15,description_w2v_16,description_w2v_17,description_w2v_18,description_w2v_19,description_w2v_20,description_w2v_21,description_w2v_22,description_w2v_23,description_w2v_24,description_w2v_25,description_w2v_26,description_w2v_27,description_w2v_28,description_w2v_29,description_w2v_30,description_w2v_31,description_w2v_32,description_w2v_33,description_w2v_34,description_w2v_35,description_w2v_36,description_w2v_37,description_w2v_38,description_w2v_39,description_w2v_40,description_w2v_41,description_w2v_42,description_w2v_43,description_w2v_44,description_w2v_45,description_w2v_46,description_w2v_47,description_w2v_48,description_w2v_49,...,PC_United States,PC_Norway,PC_Japan,PC_Switzerland,PC_India,PC_Austria,PC_Spain,PC_China,TC_etching,TC_engraving,TC_albumen print,TC_gelatin silver print,TC_letterpress printing,TC_drypoint,TC_salted paper print,TC_slide,TC_painting,TC_steel engraving,HP_0,HP_1,HP_2,color_R,color_G,color_B,color_H,color_S,color_V,plt_R,plt_G,plt_B,plt_H,plt_S,plt_V,color_count,palette_count,CL_color_b_color_g,CL_color_g_color_r,CL_color_r_color_b,CL_sum,PL_color_b_color_g,PL_color_g_color_r,PL_color_r_color_b,PL_sum,occupation_count,principal_maker_count,birth_year,death_year,maker_death_old,TE_principal_maker,TE_principal_or_first_maker
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297882,0.262706,0.191490,0.111562,0.357162,0.297882,0.316709,0.205535,0.039885,0.099732,0.874063,0.316709,4.0,8.0,0.994498,0.998758,0.988462,2.981718,0.484818,0.984007,0.405680,1.874505,3.0,1,1628,1705,77,3.433987,3.433987
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365147,0.269472,0.221796,0.055430,0.392586,0.365147,0.0,9.0,0.000000,0.000000,0.000000,0.000000,0.996004,0.986181,0.983695,2.965880,0.0,1,1864,1952,88,2.298351,2.298351
2,0.527566,-1.510556,1.498903,0.137903,-0.957198,0.802047,2.441271,-1.826873,-2.086776,-1.725259,-0.975042,0.990319,0.495875,-1.821820,2.314325,1.171428,-0.295950,-2.536932,0.561701,1.311181,1.016990,2.515146,1.519219,-1.199506,0.399765,0.149002,-1.620842,-1.061517,-0.670159,-1.629803,-0.145572,-3.111086,-0.209722,0.287282,-0.889338,-1.291739,-0.057710,-1.274380,-2.490561,-0.031008,3.078424,-1.261173,0.525886,2.090457,1.480583,-2.677032,-1.590860,-2.423737,-1.109303,-0.921814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.697569,0.651373,0.577294,0.102652,0.172420,0.697569,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.0,0.0,0.997263,0.996814,0.990159,2.984235,0.000000,0.000000,0.000000,0.000000,1.0,1,0,0,0,NaN,NaN
3,0.241789,-1.461544,-0.571965,0.051770,1.671800,-0.925241,-1.000506,0.147014,-2.080066,-0.262537,0.794321,-0.759372,1.713453,-1.130547,-0.407922,-2.496354,-0.767901,0.077382,0.731322,0.973145,-0.508279,-0.137868,-0.287884,-1.585357,0.120732,-2.239034,-1.965922,1.021036,-0.013576,-2.728281,0.682191,0.087695,0.773273,-1.560948,-1.544757,0.576463,-0.547967,-1.890046,-1.036790,0.309836,1.774340,-1.743814,0.684697,0.921820,1.642573,0.501846,1.311744,2.110542,-2.255831,-1.111799,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [6]:
test = test_X.drop(columns=['likes'])
test.shape

(12008, 371)

In [7]:
cat_params = {
    'depth' : 6,
    'learning_rate' : 0.01, 
    'eval_metric' :'RMSE', 
    'random_seed' :0,
    'num_boost_round': 10000,
}

In [8]:
categorical_features = ['LE_art_series_id', 'LE_copyright_holder', 'LE_acquisition_method',
                        'LE_principal_maker', 'LE_principal_or_first_maker',]

In [9]:
def train_catboost(X, y, params: dict, verbose=100):

    cv = KFold(n_splits=n_fold, shuffle=True, random_state=0)
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv.split(X)): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        print('x_train', x_train.shape)
        print('x_valid', x_valid.shape)
        
        train_pool = Pool(data=x_train, 
                                        label=y_train, 
                                        cat_features=categorical_features)
        validate_pool = Pool(data=x_valid, 
                                            label=y_valid, 
                                            cat_features=categorical_features)
        cat_model = CatBoostRegressor(**params)
        cat_model.fit(train_pool, eval_set=validate_pool, 
                                  verbose=verbose, 
                                  use_best_model=True,
                                  early_stopping_rounds=verbose)
        y_pred = cat_model.predict(validate_pool)
        
        oof_pred[idx_valid] = y_pred
        models.append(cat_model)
        print('Fold {} Score: {:.4f}'.format(i, np.sqrt(mean_squared_error(y_valid, y_pred))))

    score = np.sqrt(mean_squared_error(y, oof_pred))
    print('FINISHED \ whole score: {:.4f}'.format(score))
    return oof_pred, models, score

In [10]:
def train_pred(train_df, y, test_df):
    oof, models, score = train_catboost(train_df, y, cat_params)
    
    pred_list = []
    for model in models:
            test_pool = Pool(data=test_df, cat_features=categorical_features)
            pred = model.predict(test_pool)
#             pred = np.expm1(pred)
            pred_list.append(pred)
            print(pred.mean())
            
    pred = np.mean(pred_list, axis=0)
    return pred, score, models

In [11]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [12]:
%%time

pred, score, models = train_pred(train_X, target, test)

x_train (9620, 371)
x_valid (2406, 371)
0:	learn: 1.7656081	test: 1.7638279	best: 1.7638279 (0)	total: 419ms	remaining: 1h 9m 51s
100:	learn: 1.2648853	test: 1.2643817	best: 1.2643817 (100)	total: 11.8s	remaining: 19m 20s
200:	learn: 1.1272428	test: 1.1306796	best: 1.1306796 (200)	total: 24.3s	remaining: 19m 46s
300:	learn: 1.0777202	test: 1.0872014	best: 1.0872014 (300)	total: 35.8s	remaining: 19m 13s
400:	learn: 1.0511811	test: 1.0685016	best: 1.0685016 (400)	total: 48.3s	remaining: 19m 16s
500:	learn: 1.0321217	test: 1.0566834	best: 1.0566834 (500)	total: 58.9s	remaining: 18m 37s
600:	learn: 1.0161489	test: 1.0479806	best: 1.0479806 (600)	total: 1m 9s	remaining: 18m 9s
700:	learn: 1.0019120	test: 1.0404844	best: 1.0404844 (700)	total: 1m 21s	remaining: 17m 58s
800:	learn: 0.9884436	test: 1.0340679	best: 1.0340679 (800)	total: 1m 37s	remaining: 18m 40s
900:	learn: 0.9755071	test: 1.0290095	best: 1.0290095 (900)	total: 1m 49s	remaining: 18m 21s
1000:	learn: 0.9629447	test: 1.0248163	b

1600:	learn: 0.8841682	test: 0.9971317	best: 0.9971317 (1600)	total: 2m 11s	remaining: 11m 32s
1700:	learn: 0.8727334	test: 0.9952043	best: 0.9951912 (1699)	total: 2m 20s	remaining: 11m 23s
1800:	learn: 0.8622456	test: 0.9933344	best: 0.9933344 (1800)	total: 2m 27s	remaining: 11m 13s
1900:	learn: 0.8521985	test: 0.9919298	best: 0.9919059 (1899)	total: 2m 36s	remaining: 11m 4s
2000:	learn: 0.8421011	test: 0.9906970	best: 0.9906970 (2000)	total: 2m 45s	remaining: 11m
2100:	learn: 0.8328763	test: 0.9889789	best: 0.9889778 (2099)	total: 2m 54s	remaining: 10m 55s
2200:	learn: 0.8235618	test: 0.9873873	best: 0.9873873 (2200)	total: 3m 3s	remaining: 10m 48s
2300:	learn: 0.8149906	test: 0.9862383	best: 0.9862383 (2300)	total: 3m 11s	remaining: 10m 41s
2400:	learn: 0.8063180	test: 0.9850716	best: 0.9850342 (2397)	total: 3m 21s	remaining: 10m 36s
2500:	learn: 0.7979100	test: 0.9840501	best: 0.9840257 (2499)	total: 3m 30s	remaining: 10m 29s
2600:	learn: 0.7896790	test: 0.9829782	best: 0.9829782 (

3500:	learn: 0.7219668	test: 1.0280938	best: 1.0280938 (3500)	total: 5m 23s	remaining: 9m 59s
3600:	learn: 0.7151101	test: 1.0272879	best: 1.0272879 (3600)	total: 5m 32s	remaining: 9m 50s
3700:	learn: 0.7085796	test: 1.0268876	best: 1.0268876 (3700)	total: 5m 41s	remaining: 9m 41s
3800:	learn: 0.7016816	test: 1.0260243	best: 1.0260243 (3800)	total: 5m 50s	remaining: 9m 31s
3900:	learn: 0.6951657	test: 1.0254461	best: 1.0254137 (3892)	total: 5m 59s	remaining: 9m 22s
4000:	learn: 0.6889283	test: 1.0249135	best: 1.0248858 (3994)	total: 6m 8s	remaining: 9m 13s
4100:	learn: 0.6824643	test: 1.0241238	best: 1.0241238 (4100)	total: 6m 18s	remaining: 9m 3s
4200:	learn: 0.6764977	test: 1.0235146	best: 1.0235072 (4198)	total: 6m 27s	remaining: 8m 55s
4300:	learn: 0.6702790	test: 1.0228750	best: 1.0228586 (4299)	total: 6m 37s	remaining: 8m 46s
4400:	learn: 0.6646424	test: 1.0225682	best: 1.0225263 (4398)	total: 6m 46s	remaining: 8m 37s
4500:	learn: 0.6586914	test: 1.0219899	best: 1.0219899 (4500)	

2800:	learn: 0.7666617	test: 1.0031578	best: 1.0031578 (2800)	total: 4m 56s	remaining: 12m 41s
2900:	learn: 0.7588240	test: 1.0021799	best: 1.0021799 (2900)	total: 5m 5s	remaining: 12m 27s
3000:	learn: 0.7513430	test: 1.0013980	best: 1.0013980 (3000)	total: 5m 14s	remaining: 12m 14s
3100:	learn: 0.7442580	test: 1.0004610	best: 1.0004273 (3097)	total: 5m 24s	remaining: 12m 1s
3200:	learn: 0.7371304	test: 0.9998412	best: 0.9998412 (3200)	total: 5m 33s	remaining: 11m 48s
3300:	learn: 0.7298764	test: 0.9989656	best: 0.9989656 (3300)	total: 5m 42s	remaining: 11m 35s
3400:	learn: 0.7225358	test: 0.9981446	best: 0.9981422 (3395)	total: 5m 52s	remaining: 11m 23s
3500:	learn: 0.7160316	test: 0.9977064	best: 0.9976999 (3492)	total: 6m 1s	remaining: 11m 10s
3600:	learn: 0.7091944	test: 0.9970973	best: 0.9970857 (3599)	total: 6m 10s	remaining: 10m 58s
3700:	learn: 0.7026027	test: 0.9966205	best: 0.9966205 (3700)	total: 6m 20s	remaining: 10m 46s
3800:	learn: 0.6966196	test: 0.9960747	best: 0.996074

In [13]:
# feature_importance(models).head(50)

In [14]:
def revert_to_real(y_log):
    _pred = np.expm1(y_log)
    _pred = np.where(_pred < 0, 0, _pred)
    return _pred

In [15]:
test_pred = revert_to_real(pred)
print('test_pred', len(test_pred))
test_X['likes'] = test_pred
df_test_pred = test_X[['likes']]
df_test_pred.shape

test_pred 12008


(12008, 1)

In [16]:
FILENAME = 'catboost_2.csv'
OUTPUT_DIR = '../output/'
filepath = os.path.join(OUTPUT_DIR, FILENAME)
df_test_pred.to_csv(filepath, index=False)
print(filepath)

../output/catboost_2.csv


In [17]:
df_test_pred

,likes
0,1.481155
1,24.773079
2,0.581349
3,4.025066
4,0.232092
...,...
12003,0.080081
12004,0.000000
12005,0.358903
12006,0.061061


#### catboost_2: lr=0.01
- score: 0.9889
- LB = 0.9566

#### catboost_1: lr=0.1
- score: 1.0024